# Let's Load the Data

In [1]:
#I did not write this code. It was provided in the kaggle page
#https://www.kaggle.com/bryanpark/sudoku?sortBy=null&group=datasets
import numpy as np
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)
for i, line in enumerate(open('sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s
quizzes = quizzes.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))

FileNotFoundError: [Errno 2] No such file or directory: 'sudoku.csv'

# Let's Build a Basic CNN